In [55]:
import requests
import pandas as pd

# Function to fetch data from an API
def fetch_api_data(url, headers=None):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data from {url}")
        return {}

# USER CHOOSES COLLECTION
collection_slug = input("Enter the collection: ")
url = f"https://api.opensea.io/api/v2/events/collection/{collection_slug}"

# API's
url1 = f"https://api.opensea.io/api/v2/collections/{collection_slug}/stats"
url2 = f"https://api.opensea.io/api/v2/collections/{collection_slug}"
url3 = f"https://api.opensea.io/api/v2/events/collection/{collection_slug}"

headers1 = {"accept": "application/json", "x-api-key": "29b6c9b1120748878ac7f50821ec4d0b"}
headers2 = {"accept": "application/json", "x-api-key": "29b6c9b1120748878ac7f50821ec4d0b"}
headers3 = {"accept": "application/json", "x-api-key": "29b6c9b1120748878ac7f50821ec4d0b"}

data1 = fetch_api_data(url1, headers1)
data2 = fetch_api_data(url2, headers2)
data3 = fetch_api_data(url3, headers3)

Failed to fetch data from https://api.opensea.io/api/v2/collections/Mythicpunknft/stats
Failed to fetch data from https://api.opensea.io/api/v2/collections/Mythicpunknft
Failed to fetch data from https://api.opensea.io/api/v2/events/collection/Mythicpunknft


In [45]:
# Process and convert data from the first API
df1_total = pd.DataFrame([data1['total']])
df1_intervals = pd.DataFrame(data1['intervals'])

In [46]:
# Process and convert data from the second API
collection_fields = ['collection', 'name', 'description', 'image_url', 'banner_image_url', 'owner', 'category', 'is_disabled', 'is_nsfw', 'trait_offers_enabled', 'collection_offers_enabled', 'opensea_url', 'project_url', 'wiki_url', 'discord_url', 'telegram_url', 'twitter_username', 'instagram_username', 'total_supply', 'created_date']
df2_collection = pd.DataFrame([{field: data2.get(field, '') for field in collection_fields}])
df2_contracts = pd.DataFrame(data2['contracts'])
df2_fees = pd.DataFrame(data2['fees'])
df2_payment_tokens = pd.DataFrame(data2['payment_tokens'])
df2_rarity = pd.DataFrame([data2['rarity']])

In [47]:
events_normalized = []
for event in data3['asset_events']:
    # Initialize event_data with event fields, excluding nft, asset, payment, criteria
    event_data = {k: v for k, v in event.items() if k not in ['nft', 'asset', 'payment', 'criteria']}
    
    # Check and add nft data if present
    if 'nft' in event and event['nft'] is not None:
        nft_data = {'nft_' + k: v for k, v in event['nft'].items()}
        event_data.update(nft_data)
    
    # Check and add asset data if present
    elif 'asset' in event and event['asset'] is not None:
        asset_data = {'asset_' + k: v for k, v in event['asset'].items()}
        event_data.update(asset_data)
    
    # Check and include payment data if necessary and present
    if 'payment' in event and event['payment'] is not None:
        payment_data = {'payment_' + k: v for k, v in event['payment'].items()}
        event_data.update(payment_data)
    
    events_normalized.append(event_data)

df_events = pd.DataFrame(events_normalized)

In [35]:
# Export to CSV
df1_total.to_csv('api1_total_stats.csv', index=False)
df1_intervals.to_csv('api1_intervals_stats.csv', index=False)
df2_collection.to_csv('api2_collection_info.csv', index=False)
df2_contracts.to_csv('api2_collection_contracts.csv', index=False)
df2_fees.to_csv('api2_collection_fees.csv', index=False)
df2_payment_tokens.to_csv('api2_collection_payment_tokens.csv', index=False)
df2_rarity.to_csv('api2_collection_rarity.csv', index=False)
df_events.to_csv('api3_events.csv', index=False)


In [48]:
print(df_events.head())

  event_type                                         order_hash  order_type  \
0      order  0x7a9839ed655fe5c557c85827e5c1a01f5b4c5e356b12...  item_offer   
1      order  0xbca82ba0b299f59112d831114622d6d48a3cdab50528...  item_offer   
2      order  0xf18d18904e25c9287bfd42da93847111706fae1d80ed...  item_offer   
3      order  0xde2c73ad9a9c601973191f8f1360013e155df442153f...  item_offer   
4      order  0x7f5848588d4cd380cab338d3bf5ad7fe9349bd7959a8...  item_offer   

      chain                            protocol_address  start_date  \
0  ethereum  0x00000000000000adc04c56bf30ac9d3c0aaf14dc  1709141932   
1  ethereum  0x00000000000000adc04c56bf30ac9d3c0aaf14dc  1709141937   
2  ethereum  0x00000000000000adc04c56bf30ac9d3c0aaf14dc  1709141936   
3  ethereum  0x00000000000000adc04c56bf30ac9d3c0aaf14dc  1709141936   
4  ethereum  0x00000000000000adc04c56bf30ac9d3c0aaf14dc  1709141936   

   expiration_date  quantity                                       maker  \
0       1709143122    

In [49]:
print(df1_total.head())

         volume   sales  average_price  num_owners    market_cap  floor_price  \
0  1.123497e+06  109624      10.248641       11176  80293.824706       4.0699   

  floor_price_symbol  
0                ETH  


In [50]:
print(df1_intervals.head())

     interval        volume  volume_diff  volume_change  sales  sales_diff  \
0     one_day    457.523098         28.0       0.322060    112    0.333333   
1   seven_day   3704.030604       -803.0      -0.461064    899    0.000000   
2  thirty_day  22590.770628      -4037.0      -0.488392   5417    0.000000   

   average_price  
0       4.085028  
1       4.120168  
2       4.170347  


In [51]:
print(df2_fees)

   fee                                   recipient  required
0  2.5  0x0000a26b00c1f0df003000390027140000faa719      True
1  1.0  0xa858ddc0445d8131dac4d1de01f834ffcba52ef1     False


In [52]:
print(df2_rarity)


  strategy_id strategy_version               calculated_at  max_rank  \
0  openrarity          0.4.3b0  2022-09-21T08:03:09.853307     19425   

   tokens_scored  
0          19425  


In [53]:
print(df2_payment_tokens)


  symbol                                     address     chain  \
0    ETH  0x0000000000000000000000000000000000000000  ethereum   
1   WETH  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2  ethereum   
2   USDC  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48  ethereum   
3    APE  0x4d224452801aced8b2f0aebe155379bb5d594381  ethereum   

                                 image           name  decimals  \
0   https://raw.seadn.io/files/ETH.svg          Ether        18   
1  https://raw.seadn.io/files/WETH.svg  Wrapped Ether        18   
2  https://raw.seadn.io/files/USDC.svg       USD Coin         6   
3   https://raw.seadn.io/files/APE.svg        ApeCoin        18   

           eth_price                usd_price  
0  1.000000000000000  3428.269999999999982000  
1  1.000000000000000  3450.710000000000036000  
2  0.000288110000000     0.986690000000000000  
3  0.000568930000000     1.950000000000000000  


In [54]:
print(df2_collection)


              collection                   name  \
0  mutant-ape-yacht-club  Mutant Ape Yacht Club   

                                         description  \
0  The MUTANT APE YACHT CLUB is a collection of u...   

                                           image_url  \
0  https://i.seadn.io/gae/lHexKRMpw-aoSyB1WdFBff5...   

                                    banner_image_url  \
0  https://i.seadn.io/gae/5c-HcdLMinTg3LvEwXYZYC-...   

                                        owner category  is_disabled  is_nsfw  \
0  0xcac7c29bac18f080b98a3076b4e165886f509251     pfps        False    False   

   trait_offers_enabled  collection_offers_enabled  \
0                  True                       True   

                                         opensea_url project_url wiki_url  \
0  https://opensea.io/collection/mutant-ape-yacht...                        

  discord_url telegram_url twitter_username instagram_username  total_supply  \
0                                                    

In [43]:
print(df2_contracts)


                                      address     chain
0  0x60e4d786628fea6478f785a6d7e704777c86a7c6  ethereum
